# Guided Project: Profitable App Profiles for the App Store and Google Play Markets

## Introduction
The purpose of this project is to analyze data and identify categories of free apps that are likely to attract more users on both Google Play Store and App Store. For simplicity, I will focus on apps that are designed for an English-speaking audience.

I downloaded the data from Kaggle. The dataset for 10,000 Android apps from Google Play can be found [here](https://www.kaggle.com/lava18/google-play-store-apps). The dataset for 7,000 iOS apps from the App Store can be found [here](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps).

## Exploration of the Data

In [215]:
#Importing sample data of google and apple apps
open_google = open('googleplaystore.csv')
open_apple = open('AppleStore.csv')

from csv import reader

read_google = reader(open_google)
read_apple = reader(open_apple)

google_list = list(read_google)
apple_list = list(read_apple)

#Seperate header row
google_header = google_list[0]
google_apps = google_list[1:]

apple_header = apple_list[0]
apple_apps = apple_list[1:]


To browse the data, I borrowed the $\color{blue}{\text{explore_data}}$(dataset,start,end,rows_and_columns = False) custom function from Dataquest; $\color{blue}{\text{explore_data}}$ prints out user selected rows from a specified dataset and displays the number of rows and columns if $\color{red}{\text{rows_and_columns}}$ is **True**. 

In [216]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

print(google_header)   
print('\n')
explore_data(google_apps, 0,3,True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


**Tabular Description of Google Apps Columns**

|Column|Description|
|:----:|:---------:|
|App|Application Name|
|Category|Category the app belongs to|
|Rating|Overall user rating of the app(as when scraped)|
|Reviews|Number of user reviews for the app(as when scraped)|
|Size|Size of the app(as when scraped)|
|Installs|Number of user downloads/installs for the app(as when scraped|
|Type|Paid or Free|
|Price|Price of the app(as when scraped)|
|Content Rating|Age group the app is targeted at - Children/Mature 21+/Adult|
|Genres|An app can belong to multiple genres(apart from its main category)|
|Last Updated|Date when the app was last updated on Play Store(as when scraped)|
|Current Ver|Current version of the app available on Play Store(as when scraped)|
|Android Ver|Min required Android version(as when scraped)|


In [217]:
print('\n')
print(apple_header)
print('\n')
explore_data(apple_apps,0,3,True)



['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


**Tabular Description of Apple Apps Columns**

|Column|Description|
|:----:|:---------:|
|id|App ID|
|track_name|App Name|
|size_bytes|Size(in Bytes)|
|currency|Currency Type|
|price|Price Amount|
|rating_count_tot|User Rating counts(for all versions)|
|rating_count_ver|User Rating counts(for current version)|
|user_rating|Average User Rating value(for all versions)|
|user_rating_ver|Average User Rating value(for current version)|
|ver|Last Version Code|
|cont_rating|Content Rating|
|prime_genre|Primary Genre|
|sup_devices.num|Number of Supporting Devices|
|ipadSc_urls.num|Number of Screenshots Shown for Display|
|lang.num|Number of Supported Languages|
|vpp_lic|VPP Device Based Licensing Enabled|

## Data Cleaning
### Inaccurate Data
In the discussion section of the Google Play Store dataset, one of the Kaggle users points out a [mistake](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) in the entry \#10472: rating for this observation is missing. 

In [218]:
#checking if the user entry accounts for header row
print(google_apps[10472])

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


Looking at the above output, we can see that rating is not missing from the entry, but the category is not recorded which causes a shift in the columns. For this reason, I will exclude the row from the observation.

In [219]:
del google_apps[10472]

### Duplicate Google App Entries
The second issue with the Google dataset is it contains dupplicates. To verify that the issue does indeed exist, I will count them below:

In [220]:
dupplicate_g_apps = []
unique_g_apps = []

for app in google_apps:
    name = app[0]
    if name in unique_g_apps:
        dupplicate_g_apps.append(name)
    else:
        unique_g_apps.append(name)
print("The number of dupplicate Google apps is",len(dupplicate_g_apps))

The number of dupplicate Google apps is 1181


To deal with dupplicates, I will remove them and keep the observation with the largest number of reviews. The more reviews an observation has, the more recent it is.

I will start off by creating an empty dictionary. In this dictionary, I will list maximum reviews for each Google Store application.

In [221]:
#Start off by creating an empty dictionary
reviews_max = {}

for app in google_apps:
    name = app[0]
    n_reviews = float(app[3])
    if (name in reviews_max)and(reviews_max[name]<n_reviews):
        reviews_max[name] = n_reviews
    elif (name not in reviews_max):
        reviews_max[name] = n_reviews

In the next step, I create two empty lists. The first list will include information for each app with no dupplicates. The second list the names of each app in Google Play Store. 

In [222]:
#Create two new lists
android_clean = []
already_added = []

for app in google_apps:
    name = app[0]
    n_reviews = float(app[3])
    if (n_reviews == reviews_max[name])and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)
print(len(android_clean))

9659


### Non-English Speaking Apps
In the introduction, I stated that I would like to analyze only those apps that are designed for an English speaking audience. Both apple_apps and android_clean lists contain apps which are not intended for an English speaking audience. For example:

In [223]:
print(apple_apps[813][1])
print(android_clean[4412][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
中国語 AQリスニング


To exclude non-English speaking apps, I will create a custom function. In Python, each character is assigned to a number. The numbers, which correspond to the characters we use in English, are in the range 0 to 127. This information comes from ASCII (American Standard Code for Information Interchange) system. That being said, my function will check if the name of an app contains characters whose number is greater than 127 and if it does, then I will remove that app from the list as it is likely to be non-English speaking

In [224]:
def english_check(name):
    english = True 
    for character in name:
        if ord(character) > 127:
            english = False 
    return english
print(english_check('Instagram'))
print(english_check('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english_check('Docs To Go™ Free Office Suite'))
print(english_check('Instachat 😜'))

True
False
False
False


In its current state, my custom function cannot correctly detect English apps whose names include emojis and special characters. Emojis and special characters fall outside the ASCII range

In [225]:
print(ord('™'))
print(ord('😜'))

8482
128540


One solution is to exclude only those apps whose name comprises of more than 3 non-English characters. While this approach may not produce perfect results, it will minimize data loss

In [226]:
def english_check(name):
    english = True 
    count = 0
    for character in name:
        if ord(character) > 127:
            count = count + 1 
        if count > 3:
            english = False 
    return english
print(english_check('Instagram'))
print(english_check('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(english_check('Docs To Go™ Free Office Suite'))
print(english_check('Instachat 😜'))

True
False
True
True


We are now ready to filter out non-English speaking google and apple apps. 

In [227]:
clean_google = []
clean_apple = []

for app in android_clean:
    name = app[0]
    if english_check(name):
        clean_google.append(app)
for app in apple_apps:
    name = app[1]
    if english_check(name):
        clean_apple.append(app)
#number of google apps remaining        
print(len(clean_google))
#number of ios apps remaining
print(len(clean_apple))

9614
6183


### Extracting Free Apps
We are now ready to narrow down our lists to free apps. 

In [228]:
free_google = []
free_apple = []

for app in clean_google:
    price = app[7]
    if price == '0':
        free_google.append(app)
for app in clean_apple:
    price = float(app[4])
    if price == 0:
        free_apple.append(app)
print(len(free_google))
print(len(free_apple))

8864
3222


## Data Analysis
### Generate Frequency Tables
Now that the data is clean, I am ready to find the most successful genres of apps on both Google Play Store and Apple Store. To do this, I will build a frequency table for the **prime_genre** column of the Apple Store dataset and for the **Genres** and **Category** columns of the Google Play Store dataset. 

I will build 2 custom functions to generate and analyze a frequency table:

- The first function will produce a frequency table and convert frequency to percentages

- The second function will present the percentages in a descending order

In [229]:
# first function
def freq_table(dataset, index):
    dictionary = {}
    for row in dataset:
        genre = row[index]
        if genre in dictionary:
            dictionary[genre] += 1
        else:
            dictionary[genre] = 1
    for genre in dictionary:
        #transform frequency into percentage
        dictionary[genre] = (dictionary[genre]/len(dataset))*100
    return dictionary
#second function
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        #convert the dictionary into a list of tuples
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
    #organize percentages into descending order
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

### Quick Visual Scan of App Genres in App Store

In [230]:
display_table(free_apple,-5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


The most popular genre of apps in Apple Store are **Games** followed by **Entertainment** and **Photo & Video**. **Games** dominate the Apple Store market by a very tremendous margin. In terms of frequency, most of the genres are clustered close together so the standard deviation would have to be small. Since the entertainment genre captures more than 60% of the market, we can conjecture that the entertainment industry has the biggest user base. I will test this hypothesis later in the project. 

### Quick Visual Scan of App Genres in Google Play Store

In [231]:
display_table(free_google,-4)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

The first thing to notice is that the google dataset contains significantly more genres than the apple dataset. This has to do with the fact that similar genres are not aggregated into a single broad group. Given this arrangement of apps, **entertainment** industry is still the runner up in terms of popularity while **tools** sits at the top of the Google Play Store market; however, **tools** are not as astronmically popular in the Google Play Store as **games** are in the Apple Store. In terms of volume, the distribution is narrow and pretty even across the board.
### Quick Visual Scan of  App Categories in Google Play Store 

In [232]:
display_table(free_google,1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The number of categories is substantially less than the number of genres in the Google Play Store dataset. There is a significant shake up in the rankings of categories in terms of their frequency. **Family** apps overtake close to a 1/5th of Google Play Store market share. **Games** take up second biggest portion of the Google Play Store pie. 

So far, it seems like **entertainment** apps might be the most popular ones amongst users based on the numbers that exist in the Google Play Store and Apple Store space. In the next section, I will verify if that is indeed the case. 

### Most Popular Apps in Apple Store By Genre

One way to identify the genre of apps that is the most successful in the Apple Store market is to compute the average number of installs for each app genre. Unfortunately, the Apple Store dataset does not contain information on the number of installs. Yet, what I can do is use total number of user ratings(**rating_count_tot**) as a proxy. So, I will go ahead and calculate the average number of user ratings for each genre. 


In [233]:
#generating frequency table for the prime_genre column
apple_genre = freq_table(free_apple,-5)
for genre in apple_genre:
    total = 0
    len_genre = 0
    for app in free_apple:
        genre_app = app[-5]
        if genre_app == genre:
            user_ratings = float(app[5])
            total += user_ratings
            len_genre += 1
    avg_rating = total/len_genre
    print(genre,avg_rating)

Lifestyle 16485.764705882353
Travel 28243.8
News 21248.023255813954
Productivity 21028.410714285714
Navigation 86090.33333333333
Shopping 26919.690476190477
Music 57326.530303030304
Utilities 18684.456790123455
Weather 52279.892857142855
Book 39758.5
Food & Drink 33333.92307692308
Sports 23008.898550724636
Catalogs 4004.0
Finance 31467.944444444445
Education 7003.983050847458
Business 7491.117647058823
Entertainment 14029.830708661417
Social Networking 71548.34905660378
Photo & Video 28441.54375
Health & Fitness 23298.015384615384
Reference 74942.11111111111
Medical 612.0
Games 22788.6696905016


According to the results, **Navigation** is the most popular app genre on Apple Store. This comes across as a surprise. Let's take a closer look at the user ratings of every app in the genre:

In [234]:
for app in free_apple:
    genre = app[-5]
    if genre == "Navigation":
        print(app[1],":",app[5])

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


As we can see, Waze and Google Maps heavily skew the average user ratings in the positive direction for the **Navigation** apps. Thus,on average a **Navigation** app may not be the most popular one in the Apple Store after all. The second most popular apps by genre belong to **Social Networking**. Let's check it out up close:

In [235]:
for app in free_apple:
    genre = app[-5]
    if genre == "Social Networking":
        print(app[1],":",app[5])

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163
POF - Best Dating App for Conversations : 52642
Timehop : 49510
Find My Family, Friends & iPhone - Life360 Locator : 43877
Whisper - Share, Express, Meet : 39819
Hangouts : 36404
LINE PLAY - Your Avatar World : 34677
WeChat : 34584
Badoo - Meet New People, Chat, Socialize. : 34428
Followers + for Instagram - Follower Analytics : 28633
GroupMe : 28260
Marco Polo Video Walkie Talkie : 27662
Miitomo : 2

The distribution of **Social Networking** apps is uneven in terms of total user ratings: either a **Social Networking** app is extremely successful or utterly unknown. Nevertheless, I would say there is still a substantial amount of apps that are able to produce a tremendous user base. That being said, I would say on average **Social Networking** apps are the most popular ones on the Apple Store.

### Most Popular Genre Apps on Google Play Store

Unlike the Apple Store dataset, the Google Play Store dataset does record information on the number of installs. 

In [236]:
display_table(free_google,5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


The datat is not very accurate. For example, 100,000+ does not tell us whether an app has 900,000 installs or 100,000 installs; however, that is okay for what I am trying to accomplish in this project. I will leave the numbers unchanged. Currently, they are stored in the dataset as strings and I need to change them to a float type as I need to compute the average number of installs for each genre of apps.

In [237]:
google_category = freq_table(free_google,1)

#Looping over each genre in Google Play Store 
for category in google_category:
    total = 0
    len_category =0 
    for app in free_google:
        app_genre = app[1]
        if app_genre == category:
            installs = app[5]
            #removing + from installs string
            installs = installs.replace('+','')
            #removing comma from installs string
            installs = installs.replace(',','')
            total += float(installs)
            len_category += 1
    #calculating average installs for each category of apps
    avg_installs = total/len_category
    print(category,':',avg_installs)

TOOLS : 10801391.298666667
SPORTS : 3638640.1428571427
MEDICAL : 120550.61980830671
BOOKS_AND_REFERENCE : 8767811.894736841
FINANCE : 1387692.475609756
ENTERTAINMENT : 11640705.88235294
SOCIAL : 23253652.127118643
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
HOUSE_AND_HOME : 1331540.5616438356
PRODUCTIVITY : 16787331.344927534
BEAUTY : 513151.88679245283
NEWS_AND_MAGAZINES : 9549178.467741935
COMMUNICATION : 38456119.167247385
SHOPPING : 7036877.311557789
FAMILY : 3695641.8198090694
TRAVEL_AND_LOCAL : 13984077.710144928
LIBRARIES_AND_DEMO : 638503.734939759
DATING : 854028.8303030303
HEALTH_AND_FITNESS : 4188821.9853479853
EVENTS : 253542.22222222222
ART_AND_DESIGN : 1986335.0877192982
PARENTING : 542603.6206896552
PERSONALIZATION : 5201482.6122448975
AUTO_AND_VEHICLES : 647317.8170731707
PHOTOGRAPHY : 17840110.40229885
LIFESTYLE : 1437816.2687861272
FOOD_AND_DRINK : 1924897.7363636363
GAME : 15588015.603248259
MAPS_AND_NAVIGATION : 4056941.7741935486
VIDEO_PLAYERS : 247278

So on average, **Communication** apps have the highest installs on Google Play Store; however, just like with the Apple Store dataset, the number may be heavily skewed by giant apps like Facebook and Instagram.

In [239]:
len_communication = 0
for app in free_google:
    category = app[1]
    if category == "COMMUNICATION":
        len_communication += 1
print("Number of communication apps is", len_communication)
print('\n')
count = 0 
for app in free_google:
    category = app[1]
    if category == 'COMMUNICATION' and (app[5] =='50,000,000+'
                                        or app[5] == '100,000,000+'
                                        or app[5] == '500,000,000+'
                                        or app[5] == '1,000,000,000+'):
        count += 1
        print(app[0],':',app[5])
print('\n')
print("Number of large communication apps is",count)

Number of communication apps is 287


WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Contacts : 50,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
free video calls and chat : 50,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
Dolphin Browser - Fast, Private & Adblock🐬 : 50,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Mail.Ru - Email App : 50,000,000+
Hangouts : 1,000,000,000+
Azar : 50,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,00

The number of **Communication** apps that achieve more than 34 million installs on the Google Play Store is not remarkable by any means; yet, the top 34 apps in this category attract astronomical number of consumers in the market. I would argue that deciding whether **Communication** apps on average are the most successful ones in the Google Play Store space is strictly a matter of personal judgement. Personally though, I do believe that **Communication** apps do make that cut. 

## Conclusion

In this project, I cleaned and analyzed Google Play Store and Apple Store datasets to figure out which genre of apps is the most popular one among users in the  market. I arrived at the conclusion that social media genre of apps is the most popular one in the digital space; however, my findings hinge on a series of strong assumptions that I made throughout the project. If I did not make those assumptions, then my conclusions may have drastically differed.